In [1]:
import numpy as np
import scipy.sparse as sp
import os
import opt_einsum as oe
np.set_printoptions(suppress = True, linewidth = 1500, threshold = 10000, precision = 12)

# Old model periodic
from quant_rotor.core.dense.t_amplitudes_periodic import t_periodic as tpreiodic_old

# New model periodic
from quant_rotor.core.dense.t_amplitudes_periodic_fast import t_periodic as tpreiodic_dense_new
from quant_rotor.core.sparse.t_amplitudes_periodic_fast import t_periodic as tpreiodic_sparse_new_check

# Old model non-peiodic

from z_non_periodic.Non_Periodic import t_non_periodic

# New model non-periodic

from quant_rotor.core.dense.t_amplitudes_non_periodic import t_non_periodic as t_non_preiodic_dense

# Testing fast sparse and fast dense implementation for periodic systems.

In [2]:
site = 5
state = 5
g = 0.1

In [3]:
one_max_s, two_max_s, energy_s, t_a_i_tensor_s, t_ab_ij_tensor_s = tpreiodic_sparse_new_check(site, state, g, fast=True)

Done.


/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)
/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)
/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


In [4]:
one_max_d, two_max_d, energy_d, t_a_i_tensor_d, t_ab_ij_tensor_d = tpreiodic_dense_new(site, state, g, fast=True)

Done.


In [5]:
one_max_o, two_max_o, energy_o, t_a_i_tensor_o, t_ab_ij_tensor_o = tpreiodic_old(site, state, g)

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [6]:
print("Old to Sparse")
if np.array_equal(one_max_o, one_max_s):
    print("Dense:")
    print(f"One Max: {np.array_equal(one_max_o, one_max_s)}")
else:
    print(f"Delta One Max: {np.abs(one_max_o - one_max_s)}")

if np.array_equal(two_max_o, two_max_s):
    print(f"Two Max: {np.array_equal(two_max_o, two_max_s)}")
else:
    print(f"Delta Two Max: {np.abs(two_max_o - two_max_s)}")

if np.array_equal(energy_o, energy_s):
    print(f"Energy: {np.array_equal(energy_o, energy_s)}")
else:
    print(f"DeltaEnergy: {np.abs(energy_o - energy_s)}")

print(f"t_1: {np.allclose(t_a_i_tensor_o[0].reshape(state-1), t_a_i_tensor_s.toarray().reshape(state-1), 1e-10)}")
diff = np.abs(t_a_i_tensor_s.toarray().reshape(state-1) - t_a_i_tensor_o[0].reshape(state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

print(f"t_2: {np.allclose(t_ab_ij_tensor_o[0].reshape(site, state-1, state-1), np.array([i.toarray() for i in t_ab_ij_tensor_s]), 1e-10)}")
diff = np.abs(np.array([i.toarray() for i in t_ab_ij_tensor_s]) - t_ab_ij_tensor_o[0].reshape(site, state-1, state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

Old to Sparse
Dense:
One Max: True
Delta Two Max: 5.421010862427522e-19
Energy: True
t_1: True
Max absolute difference: 0.0
t_2: True
Max absolute difference: 8.673617379884035e-19


In [7]:
print("Old to Dense")

if np.array_equal(one_max_o, one_max_d):
    print("Dense:")
    print(f"One Max: {np.array_equal(one_max_o, one_max_d)}")
else:
    print(f"Delta One Max: {np.abs(one_max_o - one_max_d)}")

if np.array_equal(two_max_o, two_max_d):
    print(f"Two Max: {np.array_equal(two_max_o, two_max_d)}")
else:
    print(f"Delta Two Max: {np.abs(two_max_o - two_max_d)}")

if np.array_equal(energy_o, energy_d):
    print(f"Energy: {np.array_equal(energy_o, energy_d)}")
else:
    print(f"DeltaEnergy: {np.abs(energy_o - energy_d)}")

print(f"t_1: {np.allclose(t_a_i_tensor_o[0].reshape(state-1), t_a_i_tensor_d.reshape(state-1), 1e-10)}")
diff = np.abs(t_a_i_tensor_d.reshape(state-1) - t_a_i_tensor_o[0].reshape(state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

print(f"t_2: {np.allclose(t_ab_ij_tensor_o[0].reshape(site, state-1, state-1), t_ab_ij_tensor_d, 1e-10)}")
diff = np.abs(t_ab_ij_tensor_d - t_ab_ij_tensor_o[0].reshape(site, state-1, state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

Old to Dense
Dense:
One Max: True
Delta Two Max: 5.421010862427522e-19
Energy: True
t_1: True
Max absolute difference: 1.0842021724855044e-19
t_2: True
Max absolute difference: 1.3010426069826053e-18


In [8]:
print("Sparse to Dense")

if np.array_equal(one_max_d, one_max_s):
    print("Dense:")
    print(f"One Max: {np.array_equal(one_max_d, one_max_s)}")
else:
    print(f"Delta One Max: {np.abs(one_max_d - one_max_s)}")

if np.array_equal(two_max_d, two_max_s):
    print(f"Two Max: {np.array_equal(two_max_d, two_max_s)}")
else:
    print(f"Delta Two Max: {np.abs(two_max_d - two_max_s)}")

if np.array_equal(energy_d, energy_s):
    print(f"Energy: {np.array_equal(energy_d, energy_s)}")
else:
    print(f"DeltaEnergy: {np.abs(energy_d - energy_s)}")

print(f"t_1: {np.allclose(t_a_i_tensor_d.reshape(state-1), t_a_i_tensor_s.toarray().reshape(state-1), 1e-10)}")
diff = np.abs(t_a_i_tensor_s.toarray().reshape(state-1) - t_a_i_tensor_d.reshape(state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

print(f"t_2: {np.allclose(t_ab_ij_tensor_d.reshape(site, state-1, state-1), np.array([i.toarray() for i in t_ab_ij_tensor_s]), 1e-10)}")
diff = np.abs(np.array([i.toarray() for i in t_ab_ij_tensor_s]) - t_ab_ij_tensor_d.reshape(site, state-1, state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

Sparse to Dense
Dense:
One Max: True
Two Max: True
Energy: True
t_1: True
Max absolute difference: 1.0842021724855044e-19
t_2: True
Max absolute difference: 4.336808689942018e-19


# Check einsums.

In [9]:
p = state
a = state - 1
i = 1

A_old = np.random.rand(a, p).astype(np.float64)
B_old = np.random.rand(p, i).astype(np.float64)

B_new = B_old.reshape(p).astype(np.float64)
BB = (oe.contract("q,s->qs", B_new, B_new).reshape(p**2)).astype(np.float64)
AA =(oe.contract("ap,bq->abpq", A_old, A_old).reshape(a**2, p**2)).astype(np.float64)

In [10]:
A_old_sp = sp.csr_matrix(A_old)

B_new_sp = sp.csr_matrix(B_new.astype(np.float64)).reshape(p, 1)
BB_sp = sp.csr_matrix(BB).reshape(p**2, 1)
AA_sp = sp.csr_matrix(AA)

In [11]:
bb_sparse = B_new_sp @ B_new_sp.T

bb_term = bb_sparse.reshape(p**2, 1)

In [12]:
aa_sparse = sp.kron(A_old_sp, A_old_sp, format="csr")

In [13]:
print(np.array_equal(BB_sp.toarray(), bb_term.toarray()))
print(np.array_equal(AA_sp.toarray(), aa_sparse.toarray()))

True
True


## Residual single.

Term 1.

In [14]:
R_new = np.zeros((a), dtype=np.float64)
R_old = np.zeros((a, i), dtype=np.float64)
R_new_dot = np.zeros((a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, i), dtype=np.float64))

H_pq_old = H_pq_new = np.random.rand(p, p).astype(np.longdouble)
H_pq_old_sp = sp.csr_matrix(H_pq_old)

In [15]:
A_old_sp.shape

(4, 5)

In [16]:
R_new += oe.contract("ap,pq,q->a", A_old, H_pq_new, B_new)
R_old += oe.contract("ap,pq,qi->ai", A_old, H_pq_old, B_old)
R_new_dot += A_old @ (H_pq_old @ B_new)
R_new_dot_sp += (A_old_sp @ (H_pq_old_sp @ B_new_sp)).toarray()

print(R_new_dot_sp.shape)

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a)))

diff = np.abs(R_new - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a)))

diff = np.abs(R_new_dot - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old))

diff1 = np.abs(R_new_dot_sp - R_old)
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

(4, 1)
New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 0.0


Term 2.

In [17]:
R_new = np.zeros((a), dtype=np.float64)
R_old = np.zeros((a, i), dtype=np.float64)
R_new_dot = np.zeros((a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, i), dtype=np.float64))

T_cd_old = np.random.rand(a, a, i, i).astype(np.longdouble)
T_cd_new = T_cd_old.reshape(a**2)
T_cd_new_sp = sp.csr_matrix(T_cd_new)

V_cd_old = np.random.rand(p, i, a, a).astype(np.longdouble)
V_cd_new = V_cd_old.reshape(p, a**2)
V_cd_new_sp = sp.csr_matrix(V_cd_new)

In [18]:
R_old += oe.contract("ap,plcd,cdil->ai", A_old, V_cd_old, T_cd_old)
R_new += oe.contract("ap,pC,C->a", A_old, V_cd_new, T_cd_new)
R_new_dot += A_old @ (V_cd_new @ T_cd_new)
R_new_dot_sp += (A_old_sp @ (V_cd_new_sp @ T_cd_new_sp.T)).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a)))

diff = np.abs(R_new - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a)))

diff = np.abs(R_new_dot - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old))

diff1 = np.abs(R_new_dot_sp - R_old)
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 0.0


Term 3.

In [19]:
R_new = np.zeros((a), dtype=np.float64)
R_old = np.zeros((a, i), dtype=np.float64)
R_new_dot = np.zeros((a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, i), dtype=np.float64))

V_qq_old = np.random.rand(p, i, p, p)
V_qq_new = V_qq_old.reshape(p, p**2)
V_qq_new_sp = sp.csr_matrix(V_qq_new)

In [20]:
R_old += oe.contract("ap,plqs,qi,sl->ai", A_old, V_qq_old, B_old, B_old)
R_new += oe.contract("ap,pQ,Q->a", A_old, V_qq_new, BB)
R_new_dot += A_old @ (V_qq_new @ BB)
R_new_dot_sp += (A_old_sp @ (V_qq_new_sp @ BB_sp)).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a)))

diff = np.abs(R_new - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a)))

diff = np.abs(R_new_dot - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old))

diff1 = np.abs(R_new_dot_sp - R_old)
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 3.552713678800501e-15 

Dot:
Equal: True
Max absolute difference: 3.552713678800501e-15 

Dot Sparse:
Equal: True
Max absolute difference: 3.552713678800501e-15


## Residual doulbe sym.

Term 1.

In [71]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

V_pqrs_old = np.random.rand(p, p, p, p).astype(np.float64)
V_pqrs_new = V_pqrs_old.reshape(p**2, p**2)
V_pqrs_new_sp = sp.csr_matrix(V_pqrs_new)

In [73]:
R_old += oe.contract("ap,bq,pqrs,ri,sj->abij", A_old, A_old, V_pqrs_old, B_old, B_old)
R_new += oe.contract("BQ,QR,R->B", AA, V_pqrs_new, BB).reshape(a, a)
R_new_dot += (AA @ (V_pqrs_new @ BB)).reshape(a, a)
R_new_dot_sp += ((AA_sp @ (V_pqrs_new_sp @ BB_sp)).reshape(a, a)).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 2.842170943040401e-14 

Dot:
Equal: True
Max absolute difference: 2.842170943040401e-14 

Dot Sparse:
Equal: False
Max absolute difference: 82.93728529805153


Term 2.

In [23]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_0y_old = np.random.rand(a, a, i, i).astype(np.float64)
T_0y_new = T_0y_old.reshape(a**2)
T_0y_new_sp = sp.csr_matrix(T_0y_new)

V_pqcd_old = np.random.rand(p, p, a, a).astype(np.float64)
V_pqcd_new = V_pqcd_old.reshape(p**2, a**2)
V_pqcd_new_sp = sp.csr_matrix(V_pqcd_new)

In [24]:
R_old += oe.contract("ap,bq,pqcd,cdij->abij", A_old, A_old, V_pqcd_old, T_0y_old)
R_new += oe.contract("BQ,QC,C->B", AA, V_pqcd_new, T_0y_new).reshape(a, a)
R_new_dot += (AA @ (V_pqcd_new @ T_0y_new)).reshape(a, a)
R_new_dot_sp += (AA_sp @ (V_pqcd_new_sp @ T_0y_new_sp.T)).reshape(a, a).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 2.1316282072803006e-14 

Dot:
Equal: True
Max absolute difference: 2.1316282072803006e-14 

Dot Sparse:
Equal: True
Max absolute difference: 2.1316282072803006e-14


Term 3.

In [25]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

V_klpq_old = np.random.rand(i, i, p, p)
V_klpq_new = V_klpq_old.reshape(p, p)
V_klpq_new_sp = sp.csr_matrix(V_klpq_new)

T_0y_new_3 = T_0y_old.reshape(a, a)
T_0y_new_3_sp = sp.csr_matrix(T_0y_new_3)

In [26]:
R_old -= oe.contract("abkl,klpq,pi,qj->abij", T_0y_old, V_klpq_old, B_old, B_old)
R_new -= oe.contract("ab,pq,p,q->ab", T_0y_new_3, V_klpq_new, B_new, B_new)
R_new_dot -= T_0y_new_3 * ((V_klpq_new @ B_new) @ B_new)
R_new_dot_sp -= (T_0y_new_3_sp * ((V_klpq_new_sp @ B_new_sp).T @ B_new_sp)[0 ,0]).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 1.7763568394002505e-15


Term 4.

In [27]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_0y_new_3 = T_0y_old.reshape(a, a)
T_0y_new_3_sp = sp.csr_matrix(T_0y_new_3)

V_klcd_old = np.random.rand(i, i, a, a)
V_klcd_new = V_klcd_old.reshape(a, a)
V_klcd_new_sp = sp.csr_matrix(V_klcd_new)

In [28]:
R_old -= oe.contract("abkl,klcd,cdij->abij", T_0y_old, V_klcd_old, T_0y_old)
R_new -= oe.contract("ab,cd,cd->ab", T_0y_new_3, V_klcd_new, T_0y_new_3)
R_new_dot -= T_0y_new_3 * np.sum(V_klcd_new * T_0y_new_3)
R_new_dot_sp -= (T_0y_new_3_sp * np.sum(V_klcd_new_sp.multiply(T_0y_new_3_sp))).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 0.0


Term 5.

In [29]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

V_klcd_old = np.random.rand(i, i, a, a)
V_klcd_new = V_klcd_old.reshape(a, a)
V_klcd_new_sp = sp.csr_matrix(V_klcd_new)

T_0z_old = np.random.rand(a, a, i, i)
T_0z_new = T_0z_old.reshape(a, a)
T_0z_new_sp = sp.csr_matrix(T_0z_new)

T_yw_old = np.random.rand(a, a, i, i)
T_yw_new = T_yw_old.reshape(a, a)
T_yw_new_sp = sp.csr_matrix(T_yw_new)

In [30]:
R_old += oe.contract("klcd,acik,bdjl->abij", V_klcd_old, T_0z_old, T_yw_old)
R_new += oe.contract("cd,ac,bd->ab", V_klcd_new, T_0z_new, T_yw_new)
R_new_dot += T_0z_new @ V_klcd_new @ T_yw_new.T
R_new_dot_sp += (T_0z_new_sp @ V_klcd_new_sp @ T_yw_new_sp.T).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 4.440892098500626e-16


## Residual double sym 1.

Term 1.

In [31]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

h_pa = np.random.rand(p, a)
h_pa_new = h_pa
h_pa_new_sp = sp.csr_matrix(h_pa)

T_xy_old = np.random.rand(a, a, i, i)
T_xy_new = T_xy_old.reshape(a, a)
T_xy_new_sp = sp.csr_matrix(T_xy_new)

In [32]:
R_old += oe.contract("ap,pc,cbij->abij", A_old, h_pa, T_xy_old)
R_new += oe.contract("ap,pc,cb->ab", A_old, h_pa, T_xy_new)
R_new_dot += A_old @ h_pa_new @ T_xy_new
R_new_dot_sp += (A_old_sp @ h_pa_new_sp @ T_xy_new_sp).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 8.881784197001252e-16 

Dot Sparse:
Equal: True
Max absolute difference: 8.881784197001252e-16


Term 2.

In [33]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

h_ip_old = np.random.rand(i, p)
h_ip_new = h_ip_old.reshape(p)
h_ip_new_sp = sp.csr_matrix(h_ip_new)

T_xy_old = np.random.rand(a, a, i, i)
T_xy_new = T_xy_old.reshape(a, a)
T_xy_new_sp = sp.csr_matrix(T_xy_new)

In [34]:
R_old -= oe.contract("abkj,kp,pi->abij", T_xy_old, h_ip_old, B_old)
R_new -= oe.contract("ab,p,p->ab", T_xy_new, h_ip_new, B_new)
R_new_dot -= T_xy_new * (h_ip_new @ B_new)
R_new_dot_sp -= (T_xy_new_sp * (h_ip_new_sp @ B_new_sp)[0,0]).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 2.220446049250313e-16


Term 3.

In [35]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_xz_old = np.random.rand(a, a, i, i).astype(np.float64)
T_xz_new = T_xz_old.reshape(a, a)
T_xz_new_sp = sp.csr_matrix(T_xz_new)

V_ipap_old = np.random.rand(i, p, a, p).astype(np.float64)
V_ipap_new = V_ipap_old.reshape(p, a, p)
V_ipap_new_sp = sp.csr_matrix(V_ipap_new.reshape(p*a, p))

In [36]:
R_old += oe.contract("acik,krcs,br,sj->abij", T_xz_old, V_ipap_old, A_old, B_old)
R_new += oe.contract("ac,rcs,br,s->ab", T_xz_new, V_ipap_new, A_old, B_new)
R_new_dot += T_xz_new @ (A_old @ (V_ipap_new.reshape(p*a, p) @ B_new).reshape(p, a)).T
R_new_dot_sp += (T_xz_new_sp @ (A_old_sp @ (V_ipap_new_sp @ B_new_sp).reshape(p, a)).T).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 1.7763568394002505e-15 

Dot Sparse:
Equal: True
Max absolute difference: 1.7763568394002505e-15


Term 4.

In [37]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_cb_old = np.random.rand(a, a, i, i)
T_cb_new = T_cb_old.reshape(a, a)
T_cb_new_sp = sp.csr_matrix(T_cb_new)

V_piap_old = np.random.rand(p, i, a, p)
V_piap_new = V_piap_old.reshape(p, a, p)
V_piap_new_sp = sp.csr_matrix(V_piap_new.reshape(p*a, p))

In [38]:
R_old += oe.contract("bq,qlds,adij,sl->abij", A_old, V_piap_old, T_cb_old, B_old)
R_new += oe.contract("bq,qds,ad,s->ab", A_old, V_piap_new, T_cb_new, B_new)
R_new_dot += T_cb_new @ (A_old @ (V_piap_new.reshape(p*a, p) @ B_new).reshape(p, a)).T
R_new_dot_sp += (T_cb_new_sp @ (A_old_sp @ (V_piap_new_sp @ B_new_sp).reshape(p, a)).T).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 3.552713678800501e-15 

Dot:
Equal: True
Max absolute difference: 3.552713678800501e-15 

Dot Sparse:
Equal: True
Max absolute difference: 3.552713678800501e-15


Term 5.

In [39]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_cb_old = np.random.rand(a, a, i, i)
T_cb_new = T_cb_old.reshape(a, a)
T_cb_new_sp = sp.csr_matrix(T_cb_new)

V_iipp_old = np.random.rand(i, i, p, p)
V_iipp_new = V_iipp_old.reshape(p, p)
V_iipp_new_sp = sp.csr_matrix(V_iipp_new)

In [40]:
R_old -= oe.contract("abkj,lkrp,pi,rl->abij", T_cb_old, V_iipp_old, B_old, B_old)
R_new -= oe.contract("ab,rp,p,r->ab", T_cb_new, V_iipp_new, B_new, B_new)
R_new_dot -= T_cb_new *((V_iipp_new @ B_new) @ B_new)
R_new_dot_sp -= (T_cb_new_sp *((V_iipp_new_sp @ B_new_sp).T @ B_new_sp)[0, 0]).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 1.7763568394002505e-15 

Dot:
Equal: True
Max absolute difference: 1.7763568394002505e-15 

Dot Sparse:
Equal: True
Max absolute difference: 1.7763568394002505e-15


In [41]:

    # # h terms
    # if np.array_equal(terms.h_pp.toarray(), qs.h_term(p, p)):
    #     print("h_pp match: True")
    # else:
    #     print("h_pp mismatch:")
    #     print(np.abs(terms.h_pp.toarray() - qs.h_term(p, p)))

    # if np.array_equal(terms.h_pa.toarray(), qs.h_term(p, a)):
    #     print("h_pa match: True")
    # else:
    #     print("h_pa mismatch:")
    #     print(np.abs(terms.h_pa.toarray() - qs.h_term(p, a)))

    # if np.array_equal(terms.h_ip.toarray(), qs.h_term(i, p).reshape(i, p)):
    #     print("h_ip match: True")
    # else:
    #     print("h_ip mismatch:")
    #     print(np.abs(terms.h_ip.toarray() - qs.h_term(i, p).reshape(i, p)))

    # # V terms
    # if np.array_equal(terms.V_pppp.toarray(), qs.v_term(p, p, p, p).reshape(p**2, p**2)):
    #     print("V_pppp match: True")
    # else:
    #     print("V_pppp mismatch:")
    #     print(np.abs(terms.V_pppp.toarray() - qs.v_term(p, p, p, p).reshape(p**2, p**2)))

    # if np.array_equal(terms.V_ppaa.toarray(), qs.v_term(p, p, a, a).reshape(p**2, a**2)):
    #     print("V_ppaa match: True")
    # else:
    #     print("V_ppaa mismatch:")
    #     print(np.abs(terms.V_ppaa.toarray() - qs.v_term(p, p, a, a).reshape(p**2, a**2)))

    # if np.array_equal(terms.V_iipp.toarray(), qs.v_term(i, i, p, p).reshape(p, p)):
    #     print("V_iipp match: True")
    # else:
    #     print("V_iipp mismatch:")
    #     print(np.abs(terms.V_iipp.toarray() - qs.v_term(i, i, p, p).reshape(p, p)))

    # if np.array_equal(terms.V_iiaa.toarray(), qs.v_term(i, i, a, a).reshape(a, a)):
    #     print("V_iiaa match: True")
    # else:
    #     print("V_iiaa mismatch:")
    #     print(np.abs(terms.V_iiaa.toarray() - qs.v_term(i, i, a, a).reshape(a, a)))

    # if np.array_equal(terms.V_piaa.toarray(), qs.v_term(p, i, a, a).reshape(p, a**2)):
    #     print("V_piaa match: True")
    # else:
    #     print("V_piaa mismatch:")
    #     print(np.abs(terms.V_piaa.toarray() - qs.v_term(p, i, a, a).reshape(p, a**2)))

    # if np.array_equal(terms.V_pipp.toarray(), qs.v_term(p, i, p, p).reshape(p, p**2)):
    #     print("V_pipp match: True")
    # else:
    #     print("V_pipp mismatch:")
    #     print(np.abs(terms.V_pipp.toarray() - qs.v_term(p, i, p, p).reshape(p, p**2)))

    # if np.array_equal(terms.V_ipap.toarray(), qs.v_term(i, p, a, p).reshape(p*a, p)):
    #     print("V_ipap match: True")
    # else:
    #     print("V_ipap mismatch:")
    #     print(np.abs(terms.V_ipap.toarray() - qs.v_term(i, p, a, p).reshape(p*a, p)))

    # if np.array_equal(terms.V_piap.toarray(), qs.v_term(p, i, a, p).reshape(p*a, p)):
    #     print("V_piap match: True")
    # else:
    #     print("V_piap mismatch:")
    #     print(np.abs(terms.V_piap.toarray() - qs.v_term(p, i, a, p).reshape(p*a, p)))

In [42]:
# tensors.t_a_i_tensor, tensors.t_ab_ij_array = tensors.t_a_i_tensor.toarray().reshape(a), np.array([mat.toarray() for mat in tensors.t_ab_ij_array])

# # Check a_term
# if np.allclose(terms.a_term.toarray(), qs.A_term(a)):
#     print("a_term match: True")
# else:
#     print("a_term mismatch:")
#     print(np.abs(terms.a_term.toarray() - qs.A_term(a)))

# # Check b_term
# if np.allclose(terms.b_term.toarray(), qs.B_term(i).reshape(p, i)):
#     print("b_term match: True")
# else:
#     print("b_term mismatch:")
#     print(np.abs(terms.b_term.toarray() - qs.B_term(i).reshape(-1, 1)))

# # Check bb_term
# bb_dense = np.outer(qs.B_term(i), qs.B_term(i)).reshape(p**2)
# if np.allclose(terms.bb_term.toarray().flatten(), bb_dense):
#     print("bb_term match: True")
# else:
#     print("bb_term mismatch:")
#     print(np.abs(terms.bb_term.toarray().flatten() - bb_dense))

# # Check aa_term
# aa_dense = np.einsum("ap,bq->abpq", qs.A_term(a), qs.A_term(a)).reshape(a**2, p**2)
# if np.allclose(terms.aa_term.toarray(), aa_dense):
#     print("aa_term match: True")
# else:
#     print("aa_term mismatch:")
#     print(np.abs(terms.aa_term.toarray() - aa_dense))

# tensors.t_a_i_tensor, tensors.t_ab_ij_array = sp.csr_matrix(tensors.t_a_i_tensor), np.array(
#     [sp.csr_matrix(tensors.t_ab_ij_array[i]) for i in range(site)],dtype=object)

# Design v_full

In [43]:
state = p = 5
a = p-1
i = 1

V_tensor = np.random.rand(p, p, p, p)

In [44]:
i = 1
j = 2
k = 3
l = 4

d1 = i*state + j
d2 = k*state + l

print(V_tensor[i, j, k, l])
print(V_tensor.reshape(p**2, p**2)[d1, d2])

0.2818533272042335
0.2818533272042335


In [45]:
k_ind = np.arange(1, 5)
l_ind = np.arange(1, 5)

In [46]:
indecies = np.zeros(a**2, dtype=int)
index = 0
for k in k_ind:
    for l in l_ind:
        indecies[index] = int(k*state + l)
        index += 1

In [47]:
V_tensor.reshape(p**2, p**2)[0:1, indecies].reshape(a, a)

array([[0.173250836539, 0.81129440796 , 0.74031978218 , 0.876919677111],
       [0.856010588893, 0.224905230425, 0.153140575889, 0.748739496207],
       [0.18616102657 , 0.31709994312 , 0.706798321178, 0.151697001397],
       [0.652098550085, 0.461367561906, 0.917824129276, 0.363572780164]])

In [48]:
np.array_equal(V_tensor[0, 0, 1:5, 1:5], V_tensor.reshape(p**2, p**2)[0:1, indecies].reshape(a, a))

True

In [49]:
def v_term(v_upper_1, v_upper_2, v_lower_1, v_lower_2):
    a_v_shift = [i if a_check == a else 0 for a_check in (v_upper_1, v_upper_2, v_lower_1, v_lower_2)]
    return V_tensor[a_v_shift[0]:v_upper_1 + a_v_shift[0], a_v_shift[1]:v_upper_2 + a_v_shift[1], a_v_shift[2]:v_lower_1 + a_v_shift[2],a_v_shift[3]:v_lower_2 + a_v_shift[3]]

In [50]:
V_pppp=v_term(p, p, p, p).reshape(p**2, p**2) # 0, 0, 0, 0 -> 0:p, 0:p, 0:p, 0:p
V_ppaa=v_term(p, p, a, a).reshape(p**2, a**2) # 0, 0, i, i -> 0:p, 0:p, i:p, i:p
V_iipp=v_term(i, i, p, p).reshape(p, p) # 0, 0, 0, 0 -> 0:i, 0:i, 0:p, 0:p
V_iiaa=v_term(i, i, a, a).reshape(a, a) # 0, 0, i, i -> 0:i, 0:i, i:p, i:p
V_piaa=v_term(p, i, a, a).reshape(p, a**2) # 0, 0, i, i -> 0:p, 0:i, i:p, i:p
V_pipp=v_term(p, i, p, p).reshape(p, p**2) # 0, 0, 0, 0 -> 0:p, 0:i, 0:p, 0:p
V_ipap=v_term(i, p, a, p).reshape(p*a, p) # 0, 0, i, 0 -> 0:i, 0:p, i:p, 0:p
V_piap=v_term(p, i, a, p).reshape(p*a, p) # 0, 0, i, 0 -> 0:p, 0:i, i:p, 0:p

In [51]:
sp.csr_array([1, 2, 0, 0, 3, 0, 0, 0])

<Compressed Sparse Row sparse array of dtype 'int64'
	with 3 stored elements and shape (8,)>

In [52]:
V_matrix = sp.csr_matrix(V_tensor.reshape(p**2, p**2))

In [53]:
# csr_mat = sp.csr_matrix((data, (row_ind, col_ind)), shape=shape)

In [54]:
    # v_4_start = [i if a_check == a else 0 for a_check in (v_upper_1, v_upper_2, v_lower_1, v_lower_2)]
    # v_4_end = [v_4_start[i] + val for i, val in enumerate([v_upper_1, v_upper_2, v_lower_1, v_lower_2])]

    # rows, colums = V_matrix.nonzero()
    # data = np.array(V_matrix.data)

In [55]:
def v_term_sparse(v_u1, v_u2, v_l1, v_l2):

    # Determine shift (offset) for each index depending on whether
    # it belongs to the 'a' space (start after 'i') or 'i' (start at 0)
    def shift(size): return i if size == a else 0

    u1_shift = shift(v_u1)
    u2_shift = shift(v_u2)
    l1_shift = shift(v_l1)
    l2_shift = shift(v_l2)

    # Construct row and column index ranges
    row_indices = [
        (u1_shift + u1) * state + (u2_shift + u2)
        for u1 in range(v_u1) for u2 in range(v_u2)
    ]
    col_indices = [
        (l1_shift + l1) * state + (l2_shift + l2)
        for l1 in range(v_l1) for l2 in range(v_l2)
        ]
    return V_matrix[np.ix_(row_indices, col_indices)]

In [56]:
v_term_sparse(p, i, a, a).toarray()

array([[0.173250836539, 0.81129440796 , 0.74031978218 , 0.876919677111, 0.856010588893, 0.224905230425, 0.153140575889, 0.748739496207, 0.18616102657 , 0.31709994312 , 0.706798321178, 0.151697001397, 0.652098550085, 0.461367561906, 0.917824129276, 0.363572780164],
       [0.097972935257, 0.281732800707, 0.138242998165, 0.468583800982, 0.502399604092, 0.223377109905, 0.029034524424, 0.275633024176, 0.641659897079, 0.141646803108, 0.301363157353, 0.479200846649, 0.976492139147, 0.354731044675, 0.12359669118 , 0.959611158371],
       [0.998866198497, 0.613795214907, 0.777379646842, 0.411209113654, 0.468940869333, 0.773948932541, 0.704326199366, 0.859326781285, 0.306846052793, 0.611697839036, 0.770676609413, 0.557214402579, 0.784660774107, 0.971870213721, 0.558689862458, 0.589975460249],
       [0.655792908824, 0.293791490858, 0.78092917087 , 0.039070479486, 0.852284397681, 0.817093995786, 0.696528956891, 0.60862917045 , 0.622974718118, 0.011841401567, 0.543449253486, 0.69815374144 , 0.227

In [57]:
V_pppp=v_term_sparse(p, p, p, p) # 0, 0, 0, 0 -> 0:p, 0:p, 0:p, 0:p

if np.array_equal(V_pppp.toarray(), v_term(p, p, p, p).reshape(p**2, p**2)):
    print("V_pppp match: True")
else:
    print("V_pppp mismatch:")
    print(np.abs(V_pppp.toarray() - v_term(p, p, p, p).reshape(p**2, p**2)))

V_pppp match: True


In [58]:
V_ppaa=v_term_sparse(p, p, a, a) # 0, 0, i, i -> 0:p, 0:p, i:p, i:p

if np.array_equal(V_ppaa.toarray(), v_term(p, p, a, a).reshape(p**2, a**2)):
    print("V_ppaa match: True")
else:
    print("V_ppaa mismatch:")
    print(np.abs(V_ppaa.toarray() - v_term(p, p, a, a).reshape(p**2, a**2)))

V_ppaa match: True


In [59]:
V_iipp=v_term_sparse(i, i, p, p).reshape(p, p) # 0, 0, 0, 0 -> 0:i, 0:i, 0:p, 0:p

if np.array_equal(V_iipp.toarray(), v_term(i, i, p, p).reshape(p, p)):
    print("V_iipp match: True")
else:
    print("V_iipp mismatch:")
    print(np.abs(V_iipp.toarray() - v_term(i, i, p, p).reshape(p, p)))

V_iipp match: True


In [60]:
V_iiaa=v_term_sparse(i, i, a, a).reshape(a, a) # 0, 0, i, i -> 0:i, 0:i, i:p, i:p

if np.array_equal(V_iiaa.toarray(), v_term(i, i, a, a).reshape(a, a)):
    print("V_iiaa match: True")
else:
    print("V_iiaa mismatch:")
    print(np.abs(V_iiaa.toarray() - v_term(i, i, a, a).reshape(a, a)))

V_iiaa match: True


In [61]:
V_piaa=v_term_sparse(p, i, a, a) # 0, 0, i, i -> 0:p, 0:i, i:p, i:p

if np.array_equal(V_piaa.toarray(), v_term(p, i, a, a).reshape(p, a**2)):
    print("V_piaa match: True")
else:
    print("V_piaa mismatch:")
    print(np.abs(V_piaa.toarray() - v_term(p, i, a, a).reshape(p, a**2)))

V_piaa match: True


In [62]:
V_pipp=v_term_sparse(p, i, p, p) # 0, 0, 0, 0 -> 0:p, 0:i, 0:p, 0:p

if np.array_equal(V_pipp.toarray(), v_term(p, i, p, p).reshape(p, p**2)):
    print("V_pipp match: True")
else:
    print("V_pipp mismatch:")
    print(np.abs(V_pipp.toarray() - v_term(p, i, p, p).reshape(p, p**2)))

V_pipp match: True


In [63]:
V_ipap=v_term_sparse(i, p, a, p).reshape(p*a, p) # 0, 0, i, 0 -> 0:i, 0:p, i:p, 0:p

if np.array_equal(V_ipap.toarray(), v_term(i, p, a, p).reshape(p*a, p)):
    print("V_ipap match: True")
else:
    print("V_ipap mismatch:")
    print(np.abs(V_ipap.toarray() - v_term(i, p, a, p).reshape(p*a, p)))

V_ipap match: True


In [64]:
V_piap=v_term_sparse(p, i, a, p).reshape(p*a, p) # 0, 0, i, 0 -> 0:p, 0:i, i:p, 0:p

if np.array_equal(V_piap.toarray(), v_term(p, i, a, p).reshape(p*a, p)):
    print("V_piap match: True")
else:
    print("V_piap mismatch:")
    print(np.abs(V_piap.toarray() - v_term(p, i, a, p).reshape(p*a, p)))

V_piap match: True


# Testing old dense implementation for non-periodic system.

In [65]:
site = 3
state = 11
g = 0.1

In [66]:
one_max_o, two_max_o, energy_o, t_a_i_tensor_o, t_ab_ij_tensor_o = t_non_periodic()

Doubles: True
[[[[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]]


 [[[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]]


 [[[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]]


 [[[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]]


 [[[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]]


 [[[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]]


 [[[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0.j]]

  [[0.+0

In [67]:
one_max_d, two_max_d, energy_d, t_a_i_tensor_d, t_ab_ij_tensor_d = t_non_preiodic_dense(site, state, g)

In [68]:
print("Old to Dense")

if np.array_equal(one_max_o, one_max_d):
    print("Dense:")
    print(f"One Max: {np.array_equal(one_max_o, one_max_d)}")
else:
    print(f"Delta One Max: {np.abs(one_max_o - one_max_d)}")

if np.array_equal(two_max_o, two_max_d):
    print(f"Two Max: {np.array_equal(two_max_o, two_max_d)}")
else:
    print(f"Delta Two Max: {np.abs(two_max_o - two_max_d)}")

if np.array_equal(energy_o, energy_d):
    print(f"Energy: {np.array_equal(energy_o, energy_d)}")
else:
    print(f"DeltaEnergy: {np.abs(energy_o - energy_d)}")

print(f"t_1: {np.allclose(t_a_i_tensor_o, t_a_i_tensor_d, 1e-10)}")
diff = np.abs(t_a_i_tensor_d - t_a_i_tensor_o)
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

print(f"t_2: {np.allclose(t_ab_ij_tensor_o, t_ab_ij_tensor_d, 1e-10)}")
diff = np.abs(t_ab_ij_tensor_d - t_ab_ij_tensor_o)
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

Old to Dense
Delta One Max: 4.440284945284034e-14
Delta Two Max: 23.5266534076817
DeltaEnergy: 0.025161052210980625
t_1: False
Max absolute difference: 0.9950863729933882
t_2: False
Max absolute difference: 0.9985620104659119


In [69]:
t_a_i_tensor_d

array([[[ 0.            +0.j],
        [ 0.            +0.j],
        [-0.000950327578+0.j],
        [-0.000950327578+0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.000000037132+0.j],
        [ 0.000000037132+0.j],
        [ 0.            +0.j],
        [ 0.            +0.j]],

       [[ 0.            +0.j],
        [ 0.            +0.j],
        [-0.000937007107+0.j],
        [-0.000937007107+0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.000000036549+0.j],
        [ 0.000000036549+0.j],
        [ 0.            +0.j],
        [ 0.            +0.j]],

       [[ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j],
        [ 0.            +0.j]]])

In [70]:
t_a_i_tensor_o

array([[[0.954097519238],
        [0.277699991675],
        [0.994136045416],
        [0.193133555261],
        [0.258198204752],
        [0.816943395767],
        [0.293780805366],
        [0.30842597667 ],
        [0.859877086641],
        [0.314553482895]],

       [[0.757975767481],
        [0.193918160672],
        [0.857916447259],
        [0.958997856676],
        [0.420466828172],
        [0.902438356612],
        [0.109182944927],
        [0.865821767313],
        [0.732499134518],
        [0.410428746452]],

       [[0.932247140064],
        [0.038463589535],
        [0.31023101248 ],
        [0.35163899131 ],
        [0.236096894852],
        [0.468089609607],
        [0.217398154464],
        [0.326194621361],
        [0.254940538887],
        [0.313310347087]]])